In [1]:
import pandas as pd
import numpy as np
import gradio as gr

# utils...
import json
import requests
import random
import time

import spacy #spacy for quick language prepro
nlp = spacy.load('da_core_news_md') #instantiating English module

from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model

/Users/tobiasmjensen/anaconda3/envs/bds-chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filepath = '/Users/tobiasmjensen/Documents/aau_bds/M3 - Semester Project/app/convo_snt_chatbot.json'
data = json.load(open(filepath,'r'))

In [3]:
def text_prepro(texts):
  """
  takes in a list/iterable of texts
  removes twitter stuff
  lowercases, normalizes text
  """

  clean_container = []

  for text in nlp.pipe(texts, disable=["parser", "ner"]):

    txt = [token.lemma_.lower() for token in text # lemmatize and lower
          if token.is_alpha # remove numbers
          and not token.is_punct] # remove punctoation

    clean_container.append(" ".join(txt))

  return clean_container

In [4]:
training = []

for i in data['intents']:
  l = len(i['patterns'])
  tuples = list(zip(i['patterns'], l*[i['tag']]))
  training.extend(tuples)

In [5]:
training_df = pd.DataFrame(training, columns=['txt','label'])
training_df.head()

,txt,label
0,Jeg vil gerne bestille en vare til hjemmelevering,Bestilling af vare i butik
1,Kan jeg bestille produkter til levering?,Bestilling af vare i butik
2,Hvordan bestiller jeg varer til levering?,Bestilling af vare i butik
3,Bestilling af vare til hjemmelevering,Bestilling af vare i butik
4,Hvordan bestiller jeg produkter fra butikken?,Bestilling af vare i butik


In [6]:
training_df.txt_p = text_prepro(training_df.txt)

/var/folders/hn/x1r36wjd33b11q4x03t54rh40000gn/T/ipykernel_25423/713770530.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  training_df.txt_p = text_prepro(training_df.txt)


In [7]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
cls = LogisticRegression()

pipe = make_pipeline(tfidf, cls)
pipe.fit(training_df.txt_p, training_df.label)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [8]:
pipe.predict(['Hvordan indtaster jeg rabatkode?'])

array(['Bestilling af vare i butik'], dtype=object)

In [9]:
bot_name = '💬 SNT Bot'

In [10]:
def reply(txt):
  clean_text = text_prepro([txt])
  tag = pipe.predict(clean_text)[0]
  for intent in data['intents']:
    if tag == intent["tag"]:
      print(f"{bot_name}: {random.choice(intent['responses'])}")

In [11]:
reply('Hvordan bruger jeg en rabatkode?')

💬 SNT Bot: Når du har bestilt produkter i butikken til hjemmelevering, skal du være opmærksom på følgende:
 Så snart bestillingen er blevet gennemført i butikken, vil du både få en fysisk kvittering samt en kopi pr. e-mail. Herefter vil ordren blive pakket på vores lager og du vil efter afsendelse af ordren modtage et link på mail til trackingen af din ordre. Denne e-mail har GLS som afsender.
Som udgangspunkt vil du modtage din bestilling inden for 2-5 hverdage på den adresse, som du oplyste ved bestilling. Inden levering af pakken vil du også få mulighed for at ændre levering til en pakkeshop. Dette sker gennem den e-mail/SMS, du modtager fra GLS.


In [12]:
def reply_da(txt):
  clean_text = text_prepro([txt])
  tag = pipe.predict(clean_text)[0]
  for intent in data['intents']:
    if tag == intent["tag"]:
      return f"{bot_name}: {random.choice(intent['responses'])}"

In [13]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        # use the reply_da function to get the bot's message
        bot_message = reply_da(message)
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
